In [15]:
# Imports
from datetime import datetime
import os
import sys

import torch
import torch.nn as nn

sys.path.insert(0, os.path.abspath('..'))

BASE_PATH = os.path.dirname(sys.path[0])


from CUB.models import Multimodel
from CUB.inference import eval
from CUB.configs import multi_inst_cfg
from CUB.dataset import load_data

BASE_PATH

'/Users/hoagycunningham'

In [29]:
# Setting up

# List the model paths
# May not want to load all models into RAM simultaneously
model_with_two_joints_path = BASE_PATH + "/big_run/multi_inst_joint/20230302-141428/final_model.pth"
seq_model_path = BASE_PATH + "/big_run/seq_inst/20230224-124623/final_model.pth"
seq_sparse_model_path = BASE_PATH + "big_run/seq_inst_sparse/20230227-183548/final_model.pth"
multi_path = BASE_PATH + "/big_run/multimodel_inst/20230224-172742/final_model.pth"
ind_path = BASE_PATH + "/big_run/ind_inst/20230224-135153/final_model.pth"
                        
train_data_path = BASE_PATH + "/CUB_instance_masked/train.pkl"
val_data_path = BASE_PATH + "/CUB_instance_masked/val.pkl"
test_data_path = BASE_PATH + "/CUB_instance_masked/test.pkl"

args = multi_inst_cfg

# Looking at each model's activations

train_loader = load_data([train_data_path], args)
val_loader = load_data([val_data_path], args)
test_loader = load_data([test_data_path], args)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/hoagycunningham/CUB_instance_masked/train.pkl'

In [14]:
# Get attribute vectors across different models

ind_inst = []
seq_attrs = []
seq_sparse_attrs = []
joint_attrs = []
multi_attrs = []

model_paths = [seq_model_path, seq_sparse_model_path, model_with_two_joints_path, multi_path]
output_lists = [seq_attrs, seq_sparse_attrs, joint_attrs, multi_attrs]
with torch.no_grad():
    for model_path, olist in zip(model_paths, output_lists):
        print(model_path)
        model = torch.load(model_path)
        model.eval()
        for batch in train_loader:
            inputs, class_labels, attr_labels, attr_mask = batch

            attr_labels = [i.float() for i in attr_labels]
            attr_labels = torch.stack(attr_labels, dim=1)

            attr_labels = attr_labels.cuda() if torch.cuda.is_available() else attr_labels
            inputs = inputs.cuda() if torch.cuda.is_available() else inputs
            class_labels = class_labels.cuda() if torch.cuda.is_available() else class_labels
            attr_mask = attr_mask.cuda() if torch.cuda.is_available() else attr_mask

            output = model.generate_predictions(inputs, attr_labels, attr_mask)
            olist.append(output[0])

    del(model) # Clear memory
    print(f"Done with model {model_path.split('/')[-3]}")

/big_run/seq_inst/20230224-124623/final_model.pth


FileNotFoundError: [Errno 2] No such file or directory: '/big_run/seq_inst/20230224-124623/final_model.pth'

In [ ]:
# Looking at the difference between the two joint models
seq_attrs = seq_attrs[0]
seq_sparse_attrs = seq_sparse_attrs[0]
joint_attrs1 = joint_attrs[0]
joint_attrs2 = joint_attrs[1]
multi_attrs1 = multi_attrs[0]
multi_attrs2 = multi_attrs[1]

diff1 = joint_attrs1 - seq_attrs
diff2 = joint_attrs2 - seq_attrs

cos_sim = nn.CosineSimilarity(dim=1, eps=1e-6)

print("Cosine similarity between joint1 and seq")
print(cos_sim(joint_attrs1, seq_attrs).mean())
print("Cosine similarity between joint2 and seq")
print(cos_sim(joint_attrs2, seq_attrs).mean())
print("Cosime similiary of the deviations from seq")
print(cos_sim(diff1, diff2).mean())
print("Cosine similarity of seq and seq_sparse")
print(cos_sim(seq_attrs, seq_sparse_attrs).mean())